In [16]:
import os
import pandas as pd
import requests
import json

In [8]:
def get_embedding(text):
    # 定义你的API端点URL
    url = 'http://127.0.0.1:8888/embed'
    
    # 准备要发送的JSON数据（Python字典）
    data = {
        'texts': text,
    }

    # 发送POST请求，直接传递Python字典到json参数中
    response = requests.post(url, json=data)
    
    # 打印响应内容
    #print(response.text)
    return response.text

In [5]:
file="tariff_personal_c_MinerU.xlsx"
data=pd.read_excel(file)
data.head()

,header,data_type,context
0,恒生财富管理及個人銀行業務服務費用簡介,text,恒生财富管理及個人銀行業務服務費用簡介:2025年7月1日生效
1,恒生财富管理及個人銀行業務服務費用簡介,text,恒生财富管理及個人銀行業務服務費用簡介:A. 存款戶口及服務
2,恒生财富管理及個人銀行業務服務費用簡介,text,恒生财富管理及個人銀行業務服務費用簡介:B. 與款及找換服務
3,恒生财富管理及個人銀行業務服務費用簡介,text,恒生财富管理及個人銀行業務服務費用簡介:C. 證券、非上市債券、結構式產品、基金投資及其他投...
4,恒生财富管理及個人銀行業務服務費用簡介,text,恒生财富管理及個人銀行業務服務費用簡介:D. 卡服務


In [ ]:
writeline=[]

for i in range(data.shape[0]):
    header=data.at[i,'header']
    context=data.at[i,'context']
    embedding=json.loads(get_embedding(context))['embeddings']
    #生成写入结构
    writeline.append({
                      "header":header[0:8],
                      "context":context,
                      "vector":embedding})



In [76]:

from pymilvus import MilvusClient,connections, db

client = MilvusClient(url='http://127.0.0.1:19530',token="root:Milvus", db_name="hsbc_c_db")


res = client.list_collections()

print(res)

res = client.describe_collection(
    collection_name="hsbc_c"
)

print(res)

res = client.insert(
    collection_name='hsbc_c',
    data=writeline
)

print(res)


['hsbc_c']
{'collection_name': 'hsbc_c', 'auto_id': True, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'auto_id': True, 'is_primary': True}, {'field_id': 101, 'name': 'header', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 64}}, {'field_id': 102, 'name': 'context', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535, 'enable_match': 'true', 'enable_analyzer': 'true'}}, {'field_id': 103, 'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}], 'functions': [], 'aliases': [], 'collection_id': 460448180289679543, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': False, 'created_timestamp': 460448828784902155}
{'insert_count': 155, 'ids': [460451508745667098, 460451508745667099, 460451508745667100, 460451508745667101, 460451508745667102, 460451508745667103